### Archived Statements - part5

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516 + 2,159,986,344 + 2,159,986,344 + 2,159,986,344 = 9,719,938,548

 
Process:

 - bulk import via stream consumer of import/statements_archive5
 - use externalScroller to bulk index at the same time

#### Run the imports

In [ ]:
!import.sh -o import -t 24 -l import/statements_archive5 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step5.log

Started BulkIndex using ExternalScroller: aa27c114-50a1-452f-bb67-2f02df3381d9

In [ ]:
# check status
! cid="aa27c114-50a1-452f-bb67-2f02df3381d9"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

In [ ]:
from dateutil import parser
start = "2020-09-22T15:22:05.068Z"
end = "2020-09-24T14:54:42.148Z"
nbdocs = 2159986560
s = parser.parse(start)
e = parser.parse(end) 
throughput = nbdocs / (e-s).total_seconds()

print('%s docs/s' % format(throughput, ',.2f'))

In [ ]:
!mongo "mongodb+srv://nxbench-2826-archives.vzeqm.mongodb.net/nuxeo" \
  --username $MONGOUSER --password $MONGOPWD \
  ../ToolBox/archives-mongodb-stats.js > snapshots/"mongodb-archives-"`date +%Y%m%d-%H%M%S`.bson

In [ ]:
!statfile="snapshots/es-7B-archive-indexing-"`date +%Y%m%d-%H%M%S`.txt; \
 curl "$ES_SERVER" > $statfile; \
 curl "$ES_SERVER/_cat/health?v" >> $statfile ; \
 curl "$ES_SERVER/_cat/nodes?v" >> $statfile; \
 curl "$ES_SERVER/_cat/indices?v" >> $statfile; \
 cat $statfile